In [2]:
import os
from pathlib import Path

cwd = os.getcwd()
if cwd.split("/")[-1] == "notebooks":
    cwd = Path(cwd).parent
    os.chdir(cwd)
    print("Current working directory:", os.getcwd())

Current working directory: /Users/samlupton/Desktop/ML Projects/compromeets


# Map isochrones (skip)

In [24]:
import datetime
from pathlib import Path

import pandas as pd
import r5py
import shapely

# Load transport network
gtfs_path = Path("compromeets/artifacts/london_transport_gtfs.zip")
osm_path = Path("compromeets/artifacts/greater-london-260121.osm.pbf")

transport_network = r5py.TransportNetwork(
    osm_pbf=osm_path,
    gtfs=[
        gtfs_path,
    ],
)

# Create isochrone from a point
origin = shapely.Point(-0.1276, 51.5074)  # London coordinates

isochrones = r5py.Isochrones(
    transport_network=transport_network,
    origins=origin,
    departure=datetime.datetime(2026, 2, 1, 8, 0),  # Must match GTFS service times - using a date with active service
    transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    isochrones=[15, 30, 45],
)
print(isochrones)

isochrones_cleaned = isochrones.copy()
isochrones_cleaned["travel_time"] = (isochrones_cleaned["travel_time"].dt.total_seconds() / 60).astype(int)
isochrones_cleaned.explore(column="travel_time", cmap="YlOrRd")

      travel_time                                           geometry
0 0 days 00:15:00  MULTILINESTRING ((-0.09501 51.51255, -0.09501 ...
1 0 days 00:30:00  MULTILINESTRING ((0.00575 51.49979, 0.00575 51...
2 0 days 00:45:00  MULTILINESTRING ((0.04853 51.53863, 0.04899 51...


# Load postcode data/transport networks and calculate travel time between 2 points
This will be used to calculate the size of the isochrone needed

In [ ]:
import datetime
from pathlib import Path

import folium
import geopandas as gpd
import r5py
import shapely
from shapely.geometry import Point
from shapely.ops import unary_union

# Load transport network
gtfs_path = Path("compromeets/artifacts/london_transport_gtfs.zip")
osm_path = Path("compromeets/artifacts/greater-london-260121.osm.pbf")

transport_network = r5py.TransportNetwork(
    osm_pbf=osm_path,
    gtfs=[
        gtfs_path,
    ],
)

postcodes_gdf = pd.read_csv("~/Downloads/ONSPD_NOV_2025/Data/ONSPD_NOV_2025_UK.csv")[["pcds", "lat", "long"]]
postcodes_gdf = gpd.GeoDataFrame(postcodes_gdf, geometry=gpd.points_from_xy(postcodes_gdf.long, postcodes_gdf.lat))


def postcode_to_point(postcode, postcodes_gdf):
    return postcodes_gdf[postcodes_gdf.pcds == postcode].geometry.values[0]


def travel_time_between_postcodes(postcode1, postcode2, postcodes_gdf, transport_network):
    point1 = postcode_to_point(postcode1, postcodes_gdf)
    point2 = postcode_to_point(postcode2, postcodes_gdf)

    origins = gpd.GeoDataFrame({"id": ["origin1"]}, geometry=[point1], crs="EPSG:4326")

    destinations = gpd.GeoDataFrame({"id": ["dest1"]}, geometry=[point2], crs="EPSG:4326")

    # Calculate travel time matrix (fastest method)
    travel_times = r5py.TravelTimeMatrix(
        transport_network=transport_network,
        origins=origins,
        destinations=destinations,
        departure=datetime.datetime(2026, 2, 1, 8, 0),
        transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    )

    return int(travel_times.loc[0, "travel_time"])

/var/folders/vv/2byc96_n12g2d0tqxrl3s8sw0000gn/T/ipykernel_2437/76012361.py:20: DtypeWarning: Columns (18,31,39,44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  postcodes_gdf = pd.read_csv("~/Downloads/ONSPD_NOV_2025/Data/ONSPD_NOV_2025_UK.csv")[["pcds", "lat", "long"]]


# Calculate Isochrones

In [4]:
def postcode_to_isochrones(
    postcode,
    postcodes_gdf,
    transport_network,
    departure_time=datetime.datetime(2026, 2, 1, 8, 0),
    travel_times=[15, 30, 45],
):
    origin = postcode_to_point(postcode, postcodes_gdf)

    isochrones = r5py.Isochrones(
        transport_network=transport_network,
        origins=origin,
        departure=departure_time,  # Must match GTFS service times - using a date with active service
        transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
        isochrones=travel_times,
    )

    return isochrones, origin


isochrone_travel_time = travel_time_between_postcodes("SW1A 1AA", "N7 9RA", postcodes_gdf, transport_network) * 0.7

isochrones_person1, origin_person1 = postcode_to_isochrones(
    "SW1A 1AA", postcodes_gdf, transport_network, travel_times=[isochrone_travel_time]
)
isochrones_person2, origin_person2 = postcode_to_isochrones(
    "N7 9RA", postcodes_gdf, transport_network, travel_times=[isochrone_travel_time]
)

# Calculate and Map Overlap

In [ ]:
# Extract the 30-minute isochrone for each person
iso1 = isochrones_person1[isochrones_person1["travel_time"] == pd.Timedelta(minutes=isochrone_travel_time)]
iso2 = isochrones_person2[isochrones_person2["travel_time"] == pd.Timedelta(minutes=isochrone_travel_time)]

# Convert MULTILINESTRING to polygons by creating a buffer or using unary_union
# Method 1: If isochrones are already polygons, use them directly
# Method 2: If they're linestrings, convert to polygons


def linestring_to_polygon(gdf):
    """Convert MULTILINESTRING isochrones to polygons"""
    # Union all linestrings and create a polygon
    unioned = unary_union(gdf.geometry.values)
    # If it's already a polygon, return it
    if unioned.geom_type == "Polygon":
        return unioned
    # If it's a MultiLineString, create a buffer and then get the convex hull
    elif unioned.geom_type == "MultiLineString":
        # Small buffer to convert to polygon
        buffered = unioned.buffer(0.0001)  # Small buffer in degrees
        return buffered.convex_hull
    else:
        return unioned.convex_hull


# Convert to polygons
poly1 = linestring_to_polygon(iso1)
poly2 = linestring_to_polygon(iso2)

# Find the intersection (overlap)
overlap = poly1.intersection(poly2)

# Create a GeoDataFrame for the overlap
overlap_gdf = gpd.GeoDataFrame({"description": ["Overlap area"]}, geometry=[overlap], crs="EPSG:4326")

print(f"Overlap area: {overlap.area * 111000 * 111000:.2f} square meters")  # Rough conversion
print(f"Overlap exists: {not overlap.is_empty}")

# Find postcodes within the overlap
# Assuming you have postcodes_gdf loaded from earlier
# postcodes_gdf = gpd.GeoDataFrame(...)

# Filter postcodes that are within the overlap
postcodes_in_overlap = postcodes_gdf[postcodes_gdf.geometry.within(overlap_gdf.geometry.iloc[0])]

print(f"\nNumber of postcodes in overlap: {len(postcodes_in_overlap)}")
print(postcodes_in_overlap[["pcds", "lat", "long"]].head())

# Visualize on a map
m = folium.Map(location=[51.5074, -0.1276], zoom_start=11, tiles="OpenStreetMap")

# Add person 1 isochrone
folium.GeoJson(
    poly1,
    style_function=lambda feature: {
        "fillColor": "blue",
        "color": "blue",
        "weight": 2,
        "fillOpacity": 0.3,
    },
    tooltip="Person 1 - 30 min",
).add_to(m)

# Add person 2 isochrone
folium.GeoJson(
    poly2,
    style_function=lambda feature: {
        "fillColor": "green",
        "color": "green",
        "weight": 2,
        "fillOpacity": 0.3,
    },
    tooltip="Person 2 - 30 min",
).add_to(m)

# Add overlap area
folium.GeoJson(
    overlap,
    style_function=lambda feature: {
        "fillColor": "red",
        "color": "red",
        "weight": 3,
        "fillOpacity": 0.5,
    },
    tooltip="Overlap - Accessible to both",
).add_to(m)

# Add origin markers
folium.Marker(
    [origin_person1.y, origin_person1.x], popup="Person 1", icon=folium.Icon(color="blue", icon="user")
).add_to(m)

folium.Marker(
    [origin_person2.y, origin_person2.x], popup="Person 2", icon=folium.Icon(color="green", icon="user")
).add_to(m)

# Optionally add postcodes in overlap
if len(postcodes_in_overlap) > 0:
    for _, row in postcodes_in_overlap.head(50).iterrows():  # Limit to first 50 for performance
        folium.CircleMarker(
            [row["lat"], row["long"]], radius=3, popup=row["pcds"], color="red", fill=True, fillColor="red"
        ).add_to(m)

m

Overlap area: 3553547.17 square meters
Overlap exists: True

Number of postcodes in overlap: 4711
           pcds        lat      long
1506669  N1 9AA  51.531199 -0.122354
1506677  N1 9AL  51.530794 -0.123495
1506754  N1 9FJ  51.531123 -0.122112
1506840  N1 9NH  51.530884 -0.122900
1506851  N1 9NX  51.530884 -0.122900


# Search in Google Places API

In [6]:
centroid = overlap.centroid
center_lat = centroid.y
center_lng = centroid.x

# Calculate approximate radius in meters
# Get the distance from center to farthest point

max_distance = (
    max([Point(center_lng, center_lat).distance(Point(coord[0], coord[1])) for coord in overlap.exterior.coords])
    * 111000
)  # Rough conversion to meters (1 degree ≈ 111km)

# For Google Maps Nearby Search
location = {"latitude": center_lat, "longitude": center_lng}
radius = int(max_distance)

In [21]:
location

{'latitude': 51.52168338458761, 'longitude': -0.1267882901325607}

In [29]:
import os
from datetime import datetime

import googlemaps
import requests

API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")


gmaps = googlemaps.Client(key=API_KEY)

url = "https://places.googleapis.com/v1/places:searchNearby"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.rating,places.userRatingCount,places.location",
}
json_data = {
    "includedTypes": ["pub"],
    "maxResultCount": 10,
    "locationRestriction": {
        "circle": {"center": {"latitude": location["latitude"], "longitude": location["longitude"]}, "radius": 500.0}
    },
}

request = requests.post(url, json=json_data, headers=headers)

In [30]:
request.json()

{'places': [{'location': {'latitude': 51.52219710000001,
    'longitude': -0.1240612},
   'rating': 3.9,
   'userRatingCount': 5969,
   'displayName': {'text': 'President Hotel', 'languageCode': 'en'}},
  {'location': {'latitude': 51.524356, 'longitude': -0.1282418},
   'rating': 4,
   'userRatingCount': 4725,
   'displayName': {'text': 'Tavistock Hotel', 'languageCode': 'en'}},
  {'location': {'latitude': 51.524451199999994,
    'longitude': -0.12505339999999998},
   'rating': 4.5,
   'userRatingCount': 5838,
   'displayName': {'text': 'The Marquis Cornwallis', 'languageCode': 'en'}},
  {'location': {'latitude': 51.5181973, 'longitude': -0.1258792},
   'rating': 4.4,
   'userRatingCount': 2641,
   'displayName': {'text': 'Museum Tavern', 'languageCode': 'en'}},
  {'location': {'latitude': 51.5210983, 'longitude': -0.12288969999999999},
   'rating': 4.6,
   'userRatingCount': 2584,
   'displayName': {'text': 'Swan', 'languageCode': 'en'}},
  {'location': {'latitude': 51.5229095, 'longi

In [ ]:
import os

API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
location = {"latitude": 37.7937, "longitude": -122.3965}

In [7]:
os.getenv("GOOGLE_PLACES_API_KEY")

In [9]:
from compromeets.google_places_client import GooglePlacesClient

with GooglePlacesClient(API_KEY) as client:
    response = client.search_nearby(location, 500, ["pub"])

{'places': [{'location': {'latitude': 37.794061299999996, 'longitude': -122.401694}, 'rating': 4.3, 'userRatingCount': 1110, 'displayName': {'text': 'Irish Times', 'languageCode': 'en'}}, {'location': {'latitude': 37.7941431, 'longitude': -122.3992091}, 'rating': 4.3, 'userRatingCount': 741, 'displayName': {'text': 'Royal Exchange Restaurant', 'languageCode': 'en'}}, {'location': {'latitude': 37.794027199999995, 'longitude': -122.399087}, 'rating': 4, 'userRatingCount': 482, 'displayName': {'text': "Harrington's Bar and Grill - Irish Pub", 'languageCode': 'en'}}, {'location': {'latitude': 37.795, 'longitude': -122.39888889999997}, 'rating': 4.3, 'userRatingCount': 497, 'displayName': {'text': 'Patriot House Pub', 'languageCode': 'en'}}]}
